In [ ]:
# Always run this first before any import
# FastAI and Kaggle Setup
!curl https://course-v3.fast.ai/setup/colab | bash
! pip install kaggle

## Downloading the Data


Go to the link below, and remember to replace the **username** field below with your **kaggle username**  

https://www.kaggle.com/[username]/account

Then click on **Create New API Token**
It should have downloaded a  file called kaggle.json

Locate the kaggle.json file on your computer and upload it to this workspace (Click on files in this workspace and select upload)

After uploading the kaggle.json file, run the next line

In [ ]:
# Make a folder in the root called .kaggle and move the uploaded kaggle.json 
# file into the .kaggle directory
!mkdir ~/.kaggle/ && mv kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the data 
!kaggle competitions download -c hackexpo2018

In [ ]:
# Unzip the files into train and test directories
!unzip test.zip 
!unzip train.zip

# remove the sample-submission.csv and the zipped files
!rm -rf sample-submission.csv 
!rm -rf test.zip train.zip


In [ ]:
# Import necessary libraries
from fastai import *
from fastai.vision import * 
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
np.random.seed(42)
path = Path('train')
path.mkdir(parents=True, exist_ok=True)
src = (ImageItemList.from_folder(path, folder='train', extensions='jpg').random_split_by_pct(0.2))


In [ ]:
# Get some pre-built  transforms from FastAI library
tfms = get_transforms()
tfms

In [ ]:
# Read the Train Data, and it randomly into 80% train set 
# and 20% for validation set

data = (ImageItemList.from_folder(path)
        .random_split_by_pct(0.2)
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch())

In [ ]:
# Show some of the Images in the Dataset
data.show_batch(rows=3, figsize=(7,6))


In [ ]:
# Create a model
learn = create_cnn(data, models.resnet152, metrics=accuracy)

# Train the model for 6 epochs
learn.fit_one_cycle(9)

In [ ]:
# Get the top predictions our model got wrong
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)


In [ ]:
# Save the trained model
learn.save('model-resnet152')

In [ ]:
# Store all the names of the images in the test folder in a list 
image_names = os.listdir('test')

image_root_names = Path('test').ls()

# Get Prediction for each Image
preds = [np.int(learn.predict(open_image(image))[0]) for image in image_root_names]

In [ ]:
# Create a DataFrame for our predictions
prediction_df = pd.DataFrame({'ImageID':image_names, 'Category': preds})

In [ ]:
# Save our predictions in a csv file called submission.csv
prediction_df.to_csv('submission152.csv',index=False)

In [ ]:
# Get the class to label mapping
class_label_map = {'0': 'Sparse Traffic', '1' : 'Dense Traffic', 
                   '2': 'an Accident', '3': 'Vehicle on Fire'}

# Look at some of our predictions
prediction_df.head()

In [ ]:
 '''
 Predict a random image from a directory

'''

def show_image_pred(images_dir='test', class_map = None):
  
  image_name_list = Path(images_dir).ls()
  
  random_img = np.random.randint(len(image_name_list))
  img_path = image_name_list[random_img]
  pred = 'I predicted class {}'.format(learn.predict(open_image(img_path))[0])
  if class_map:
    pred = 'My prediction: {}'.format(class_map[learn.predict(open_image(img_path))[0]])
  plt.axis('off')
  plt.title(pred)
  plt.imshow(Image.open(img_path))
  

In [ ]:
show_image_pred(class_map=class_label_map)